# Model PEFT training

## Create LoRA model

In [5]:
import os
import sys

from modules.peft_utils import add_peft_to_model

sys.path.append(os.path.abspath(os.path.join('..')))
model_language = "french"
from modules.data import get_quebecois_data
from modules.model import get_whisper

model = get_whisper()

model.generation_config.language = model_language
model.generation_config.task = "transcribe"

model.requires_grad_(False)
lora_model = add_peft_to_model(model)
lora_model.print_trainable_parameters()


/Users/manuel/PyCharmMiscProject/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


trainable params: 3,594,240 || all params: 245,329,152 || trainable%: 1.4651


## Setup trainer

In [2]:
from pathlib import Path
from modules.training import compute_metrics, DataCollatorSpeechSeq2SeqWithPadding
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, WhisperProcessor

model_name = "openai/whisper-small"
tokenizer_language = "French"
processor = WhisperProcessor.from_pretrained(model_name, language="French", task="transcribe")

training_args = Seq2SeqTrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    gradient_checkpointing=False,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    save_strategy='best',  #Keep only best model when saving
    save_steps=500,
    save_only_model=True,
    save_total_limit=2,
    # eval_strategy="no",
    eval_strategy="steps",
    eval_steps=500,
    fp16=True,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    generation_max_length=225,
    logging_steps=100,
    report_to=[],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)
data = get_quebecois_data()
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor,
                                                     decoder_start_token_id=model.config.decoder_start_token_id)
test = data["test"].take(500)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=lora_model,
    train_dataset=data["train"],
    eval_dataset=test,
    data_collator=data_collator,
    processing_class=processor,
    compute_metrics=compute_metrics,
)
print(trainer)

ModuleNotFoundError: No module named 'modules'

In [15]:
from datetime import datetime

#Create new directory for checkpoints with the model languages
output_dir = Path(
    f'cps/cp_{model_name.replace('/', '-')}-{model_language}_T{tokenizer_language}_{datetime.now().strftime("%d-%m-%Y_%H:%M")}')
output_dir.mkdir(parents=True, exist_ok=True)
trainer.args.output_dir = str(output_dir)
trainer.train()

21_10:47


## Training

In [ ]:
trainer.train()

In [ ]:
from transformers import WhisperForConditionalGeneration
from peft import PeftModel

path_to_peft = ""
whisper = WhisperForConditionalGeneration.from_pretrained(model_name)
peft = PeftModel.from_pretrained(whisper, path_to_peft)

# model = get_peft_model(whisper, peft)